# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups?
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Import the libraries needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math

In [2]:
#loading the csv file
df = pd.read_csv("homepage_actions.csv")

In [3]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
# Count of actions by group
action_group_count = df.groupby('group')['action'].value_counts().unstack()
print("\nCount of actions by group:\n", action_group_count)


Count of actions by group:
 action      click  view
group                  
control       932  3332
experiment    928  2996


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [5]:
# Filter DataFrame to count 'click' occurrences in each group
clicks_experiment = df[(df['group'] == 'experiment') & (df['action'] == 'click')].shape[0]
views_experiment = df[df['group'] == 'experiment'].shape[0]

clicks_control = df[(df['group'] == 'control') & (df['action'] == 'click')].shape[0]
views_control = df[df['group'] == 'control'].shape[0]

# Calculate click-through rate (CTR) for each group
ctr_experiment = clicks_experiment / views_experiment
ctr_control = clicks_control / views_control

# Perform two-sample t-test
_, p_value = stats.ttest_ind(df[df['group'] == 'experiment']['action'] == 'click', df[df['group'] == 'control']['action'] == 'click')

# Check if the p-value is significant (e.g., less than 0.05)
if p_value < 0.05:
    print("The experimental homepage was more effective than the control group (statistically significant).")
else:
    print("There is no significant difference between the experimental and control groups in terms of effectiveness.")

There is no significant difference between the experimental and control groups in terms of effectiveness.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not.

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate.
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group.

In [6]:
# Calculate the click-through rate (CTR) for the control group
clicks_control = df[(df['group'] == 'control') & (df['action'] == 'click')].shape[0]
views_control = df[df['group'] == 'control'].shape[0]
ctr_control = clicks_control / views_control

# Calculate the number of views in the experiment group
views_experiment = df[df['group'] == 'experiment'].shape[0]

# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = ctr_control * views_experiment

print("Expected number of clicks for the experiment group (if CTR was the same as control group):", expected_clicks_experiment)

Expected number of clicks for the experiment group (if CTR was the same as control group): 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [7]:
# Calculate the standard deviation of the binomial distribution
std_dev = math.sqrt(views_experiment * ctr_control * (1 - ctr_control))

print("Standard deviation:", std_dev)

Standard deviation: 25.888551746650283


### Step 3:
Finally, calculate a p-value using the normal distribution based on this z-score.

In [9]:
# Calculate the actual number of clicks for the experiment group
clicks_experiment = df[(df['group'] == 'experiment') & (df['action'] == 'click')].shape[0]

# Calculate the z-score
z_score = (clicks_experiment - expected_clicks_experiment) / std_dev

# Calculate the p-value using the normal distribution
p_value = 1 - stats.norm.cdf(z_score)

print("Z-score:", z_score)
print("P-value based on the normal distribution:", p_value)

Z-score: 2.71607302278784
P-value based on the normal distribution: 0.0033030672759265522


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment:  Both analyses indicate a significant difference in click-through rates, with the experimental homepage showing higher effectiveness compared to the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.